## Extract the text

In [7]:
import whisper
# from tool_language_video_translate.utils import extract_audio_from_video
from tool_language_video_translate import utils


In [8]:
import os
video_name = "Pep_Guardiola_explained_why_Chelsea_is_so_good"
video_path = f"./video_data/{video_name}.mp4"
os.path.exists(video_path)
#/home/davidtito/Documents/Projects/tool_language_video_translate/video_data/Pep_Guardiola_explained_why_Chelsea_is_so_good.mp4

True

In [9]:
video_name = "Pep_Guardiola_explained_why_Chelsea_is_so_good"
video_path = f"./video_data/{video_name}.mp4"
audio_path = f"./video_data/origin_audio_{video_name}.wav"
utils.extract_audio(video_path=video_path, audio_path=audio_path)

ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 14 (GCC)
  configuration: --prefix=/usr --bindir=/usr/bin --datadir=/usr/share/ffmpeg --docdir=/usr/share/doc/ffmpeg --incdir=/usr/include/ffmpeg --libdir=/usr/lib64 --mandir=/usr/share/man --arch=x86_64 --optflags='-O2 -flto=auto -ffat-lto-objects -fexceptions -g -grecord-gcc-switches -pipe -Wall -Werror=format-security -Wno-error=incompatible-pointer-types -Wp,-U_FORTIFY_SOURCE,-D_FORTIFY_SOURCE=3 -Wp,-D_GLIBCXX_ASSERTIONS -specs=/usr/lib/rpm/redhat/redhat-hardened-cc1 -fstack-protector-strong -specs=/usr/lib/rpm/redhat/redhat-annobin-cc1 -m64 -march=x86-64 -mtune=generic -fasynchronous-unwind-tables -fstack-clash-protection -fcf-protection -mtls-dialect=gnu2 -fno-omit-frame-pointer -mno-omit-leaf-frame-pointer ' --extra-ldflags='-Wl,-z,relro -Wl,--as-needed -Wl,-z,pack-relative-relocs -Wl,-z,now -specs=/usr/lib/rpm/redhat/redhat-hardened-ld -specs=/usr/lib/rpm/redhat/redhat-annobin-cc1 -Wl,--build-id

'./video_data/origin_audio_Pep_Guardiola_explained_why_Chelsea_is_so_good.wav'

In [10]:
def transcribe_audio(audio_path, model_size="small"):
    """
    Transcribe audio using whisper.
    """
    model = whisper.load_model(model_size)
    result = model.transcribe(audio_path)
    return result["text"]

In [11]:
text = transcribe_audio(audio_path)

In [12]:
text

" Why Chelsea plays so good? Because they have three centre defender closers, the two-holding midfielders, the relating move closers, the pocket closers, they have structured the five and two players so, so close. And the distances are so close. And at the same time, they are so wide with the wind backs, they are so deep with the Burnler mookers in behind. That's why you cannot be close because they push you here, push you here, and a lot of good players in the middle. And that's why it's difficult to, you know, to face the teams like they want the ball, they want to play with the ball. And we try to do it since the day one we arrived here. You've moved nicely onto Chelsea."

### Translate text

In [13]:
# import request
# from config import DEEP_API_KEY

# def translate_text(text, target_lang="EN"):
#     """
#     Translate text to the target language using DeepL API.
#     """
#     url = "https://api-free.deepl.com/v2/translate"
#     payload = {
#         "auth_key":DEEP_API_KEY, 
#         "text":text, 
#         "target_lang":target_lang
#     }


In [ ]:
api_print = os.getenv("DEEP_API_KEY")
api_print

In [15]:
import deepl
import os
#from config import DEEP_API_KEY

def translate_text(text, target_lang, context=None):
    """
    Translate text using DeepL
    """
    DEEP_API_KEY = os.getenv("DEEP_API_KEY")
    translator = deepl.Translator(DEEP_API_KEY)

    result = translator.translate_text(text=text, 
                                        target_lang=target_lang, 
                                        context=context)
    return result.text



In [16]:
context = "This is a tactic football class video"
text_translated = translate_text(text, target_lang="PT-BR", context=context)

In [17]:
print(text)
print(text_translated)

 Why Chelsea plays so good? Because they have three centre defender closers, the two-holding midfielders, the relating move closers, the pocket closers, they have structured the five and two players so, so close. And the distances are so close. And at the same time, they are so wide with the wind backs, they are so deep with the Burnler mookers in behind. That's why you cannot be close because they push you here, push you here, and a lot of good players in the middle. And that's why it's difficult to, you know, to face the teams like they want the ball, they want to play with the ball. And we try to do it since the day one we arrived here. You've moved nicely onto Chelsea.
 Por que o Chelsea joga tão bem? Porque eles têm três zagueiros centrais mais próximos, dois meio-campistas de contenção, os que se movimentam em relação a eles, os que fecham o bolso, eles estruturaram os cinco e os dois jogadores de forma muito, muito próxima. E as distâncias são muito próximas. E, ao mesmo tempo, 

### TTS (Text To Speech)

In [18]:
import azure.cognitiveservices.speech as speechsdk

In [ ]:
AZURE_TTS_KEY = os.getenv("AZURE_TTS_KEY")
AZURE_REGION = os.getenv("AZURE_REGION")
#print(AZURE_TTS_KEY)
#print(AZURE_REGION)


In [20]:
def synthesize_speech(text:str, output_path:str, voice:str="pt-BR-MacerioMultilingualNeural"):
    """
    Convert text to speech
    """

    try:
        # Setting the auth
        AZURE_TTS_KEY = os.getenv("AZURE_TTS_KEY")
        AZURE_REGION = os.getenv("AZURE_REGION")
        speech_config = speechsdk.SpeechConfig(subscription=AZURE_TTS_KEY, region=AZURE_REGION)
        speech_config.speech_synthesis_voice_name = voice

        # Define the output
        audio_config = speechsdk.audio.AudioOutputConfig(filename=output_path)

        # Create the synthesizer
        synthesizer = speechsdk.SpeechSynthesizer(speech_config=speech_config, audio_config=audio_config)

        # Run synthesizer
        result = synthesizer.speak_text_async(text).get()

        # Check erro
        if result.reason == speechsdk.ResultReason.SynthesizingAudioCompleted:
            print(f"Audio in PT-BR created: {output_path}")
            return output_path
        elif result.reason == speechsdk.ResultReason.Canceled:
            cancellation_details = result.cancellation_details
            raise RuntimeError(f"Speech synthesis canceled: {cancellation_details.reason}. Error details: {cancellation_details.error_details}")

    except Exception as e:
        raise RuntimeError(f"An error when synthesizing speech: {e}")

In [ ]:
output_audio_path = f"./video_data/traslated_audio_{video_name}.wav"
synthesize_speech(text_translated, output_audio_path)

### Merge Audio

In [ ]:
output_video_path = f"./video_data/translated_{video_name}.mp4"
utils.merge_audio_with_video(video_path, output_audio_path, output_video_path)

ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 14 (GCC)
  configuration: --prefix=/usr --bindir=/usr/bin --datadir=/usr/share/ffmpeg --docdir=/usr/share/doc/ffmpeg --incdir=/usr/include/ffmpeg --libdir=/usr/lib64 --mandir=/usr/share/man --arch=x86_64 --optflags='-O2 -flto=auto -ffat-lto-objects -fexceptions -g -grecord-gcc-switches -pipe -Wall -Werror=format-security -Wno-error=incompatible-pointer-types -Wp,-U_FORTIFY_SOURCE,-D_FORTIFY_SOURCE=3 -Wp,-D_GLIBCXX_ASSERTIONS -specs=/usr/lib/rpm/redhat/redhat-hardened-cc1 -fstack-protector-strong -specs=/usr/lib/rpm/redhat/redhat-annobin-cc1 -m64 -march=x86-64 -mtune=generic -fasynchronous-unwind-tables -fstack-clash-protection -fcf-protection -mtls-dialect=gnu2 -fno-omit-frame-pointer -mno-omit-leaf-frame-pointer ' --extra-ldflags='-Wl,-z,relro -Wl,--as-needed -Wl,-z,pack-relative-relocs -Wl,-z,now -specs=/usr/lib/rpm/redhat/redhat-hardened-ld -specs=/usr/lib/rpm/redhat/redhat-annobin-cc1 -Wl,--build-id

'./video_data/translated_Pep_Guardiola_explained_why_Chelsea_is_so_good.mp4'

## Nova versão com timer

In [ ]:
#from faster_whisper import WhisperModel

In [2]:
from tool_language_video_translate.asr import transcribe_audio_with_segments

/home/davidtito/Documents/Projects/Tool_Translate_Language_Video/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
video_name = "Pep_Guardiola_explained_why_Chelsea_is_so_good"
video_path = f"./video_data/{video_name}.mp4"
audio_path = f"./video_data/origin_audio_{video_name}.wav"
# utils.extract_audio(video_path=video_path, audio_path=audio_path)
segmentos = transcribe_audio_with_segments(audio_path, lang_src="en", model_size="small", device="cpu")

In [4]:
segmentos

[Segment(start=0.0, end=4.88, text=['Why', 'Chelsea', 'plays', 'so', 'good?', 'Because', 'they', 'have', 'three', 'central', 'defender', 'closers,']),
 Segment(start=4.88, end=7.88, text=['the', 'two-holding', 'midfielders,', 'the', 'relating', 'move', 'closers,', 'the', 'pocket', 'closers,']),
 Segment(start=7.88, end=11.52, text=['they', 'have', 'structured', 'the', 'five', 'and', 'two', 'players', 'so,', 'so', 'close.']),
 Segment(start=11.52, end=17.6, text=['And', 'the', 'distances', 'are', 'so', 'close,', 'and', 'at', 'the', 'same', 'time,', 'they', 'are', 'so', 'wide', 'with', 'the', 'windbacks,']),
 Segment(start=17.6, end=20.52, text=['they', 'are', 'so', 'deep', 'with', 'the', 'Burnler', 'mookers', 'in', 'behind.']),
 Segment(start=20.52, end=23.8, text=["That's", 'why', 'you', 'cannot', 'be', 'close,', 'because', 'they', 'push', 'you', 'here,', 'push', 'you', 'here,']),
 Segment(start=23.8, end=25.64, text=['and', 'a', 'lot', 'of', 'good', 'players', 'in', 'the', 'middle.'])

In [5]:
list = [" ".join(s.text) for s in segmentos]
list

['Why Chelsea plays so good? Because they have three central defender closers,',
 'the two-holding midfielders, the relating move closers, the pocket closers,',
 'they have structured the five and two players so, so close.',
 'And the distances are so close, and at the same time, they are so wide with the windbacks,',
 'they are so deep with the Burnler mookers in behind.',
 "That's why you cannot be close, because they push you here, push you here,",
 'and a lot of good players in the middle.',
 "And that's why it's difficult to, you know, to face the teams like they won the ball",
 'and they want to play with the ball.',
 'And we try to do it since the day one we arrived here.',
 "You've moved nicely onto Chelsea."]

In [6]:
from tool_language_video_translate.translate import load_translation_model, translate_segments
# from transformers import MarianTokenizer, MarianMTModel

In [ ]:
context = "This is a tactic football class video"
target_lang="PT-BR"

tm = load_translation_model(lang_src="en", lang_tgt="pt")
translated = translate_segments(segments=segmentos, tm=tm, batch_size=8, target_lang=target_lang, context=context)

In [8]:
translated

# ['Por que o Chelsea joga tão bem? Porque eles têm três zagueiros centrais que fecham o gol,',
#  'os dois meio-campistas de contenção, os fechadores de jogadas relacionadas, os fechadores de bolsos,',
#  'eles estruturaram os cinco e dois jogadores de forma muito, muito próxima.',
#  'E as distâncias são tão próximas e, ao mesmo tempo, são tão amplas com os windbacks,',
#  'eles são tão profundos com os mookers do Burnler atrás.',
#  'É por isso que você não pode ficar perto, porque eles o empurram para cá, empurram para lá,',
#  'e muitos bons jogadores no meio.',
#  'E é por isso que é difícil, sabe, enfrentar as equipes como se elas tivessem ganhado a bola',
#  'e eles querem jogar com a bola.',
#  'E tentamos fazer isso desde o primeiro dia em que chegamos aqui.',
#  'Você passou muito bem para o Chelsea.']

['Por que o Chelsea joga tão bem? Porque eles têm três zagueiros centrais que fecham o gol,',
 'os dois meio-campistas de contenção, os fechadores de jogadas relacionadas, os fechadores de bolsos,',
 'eles estruturaram os cinco e dois jogadores de forma muito, muito próxima.',
 'E as distâncias são tão próximas e, ao mesmo tempo, são tão amplas com os windbacks,',
 'eles são tão profundos com os mookers do Burnler atrás.',
 'É por isso que você não pode ficar perto, porque eles o empurram para cá, empurram para lá,',
 'e muitos bons jogadores no meio.',
 'E é por isso que é difícil, sabe, enfrentar as equipes como se elas tivessem ganhado a bola',
 'e eles querem jogar com a bola.',
 'E tentamos fazer isso desde o primeiro dia em que chegamos aqui.',
 'Você passou muito bem para o Chelsea.']

In [9]:
## Build Timed Lines
from tool_language_video_translate.timespeech import build_timed_lines

In [10]:
lines = build_timed_lines(segmentos, translated)
lines

[TimedLine(start=0.0, end=4.88, text_translated='Por que o Chelsea joga tão bem? Porque eles têm três zagueiros centrais que fecham o gol,', rate_pct=122),
 TimedLine(start=4.88, end=7.88, text_translated='os dois meio-campistas de contenção, os fechadores de jogadas relacionadas, os fechadores de bolsos,', rate_pct=125),
 TimedLine(start=7.88, end=11.52, text_translated='eles estruturaram os cinco e dois jogadores de forma muito, muito próxima.', rate_pct=125),
 TimedLine(start=11.52, end=17.6, text_translated='E as distâncias são tão próximas e, ao mesmo tempo, são tão amplas com os windbacks,', rate_pct=92),
 TimedLine(start=17.6, end=20.52, text_translated='eles são tão profundos com os mookers do Burnler atrás.', rate_pct=125),
 TimedLine(start=20.52, end=23.8, text_translated='É por isso que você não pode ficar perto, porque eles o empurram para cá, empurram para lá,', rate_pct=125),
 TimedLine(start=23.8, end=25.64, text_translated='e muitos bons jogadores no meio.', rate_pct=11

In [1]:
import pickle

#with open("timed_lines.pkl", "wb") as f:
#    pickle.dump(lines, f)

with open("timed_lines.pkl", "rb") as f:
    linhas_recuperadas = pickle.load(f)
lines = linhas_recuperadas.copy()

/home/davidtito/Documents/Projects/Tool_Translate_Language_Video/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
from tool_language_video_translate.tts import synthesize_all, synthesize_segment_tts
import asyncio, nest_asyncio
import os

In [12]:
# test synthesize_segment_tts function
test_line = lines[0]
tmpdir = f"./temp/{video_name}"
outfile_test = os.path.join(tmpdir, "segment_0_test.mp3")
# Create temporary directory if it doesn't exist
os.makedirs(tmpdir, exist_ok=True)

await synthesize_segment_tts(text=test_line.text_translated, 
                        rate_pct= test_line.rate_pct,   
                        outfile=outfile_test,
                        )

In [13]:
# Test synthesize_all function
voice = 'pt-BR-MacerioMultilingualNeural'
tmpdir = f"./temp/{video_name}"
# tempdir
outfiles = await synthesize_all(lines, tmpdir=tmpdir)
# outfiles

In [14]:
outfiles

['./temp/Pep_Guardiola_explained_why_Chelsea_is_so_good/seg_00000.mp3',
 './temp/Pep_Guardiola_explained_why_Chelsea_is_so_good/seg_00001.mp3',
 './temp/Pep_Guardiola_explained_why_Chelsea_is_so_good/seg_00002.mp3',
 './temp/Pep_Guardiola_explained_why_Chelsea_is_so_good/seg_00003.mp3',
 './temp/Pep_Guardiola_explained_why_Chelsea_is_so_good/seg_00004.mp3',
 './temp/Pep_Guardiola_explained_why_Chelsea_is_so_good/seg_00005.mp3',
 './temp/Pep_Guardiola_explained_why_Chelsea_is_so_good/seg_00006.mp3',
 './temp/Pep_Guardiola_explained_why_Chelsea_is_so_good/seg_00007.mp3',
 './temp/Pep_Guardiola_explained_why_Chelsea_is_so_good/seg_00008.mp3',
 './temp/Pep_Guardiola_explained_why_Chelsea_is_so_good/seg_00009.mp3',
 './temp/Pep_Guardiola_explained_why_Chelsea_is_so_good/seg_00010.mp3']

In [2]:
#with open("path_list.pkl", "wb") as f:
#    pickle.dump(outfiles, f)

with open("path_list.pkl", "rb") as f:
    outfiles_recuperadas = pickle.load(f)
outfiles = outfiles_recuperadas.copy()

In [5]:
# união de timeline
from tool_language_video_translate.utils import build_timeline_audio

timeline = build_timeline_audio(lines=lines, seg_files=outfiles, sr=44100)

In [6]:
import os
video_name = "Pep_Guardiola_explained_why_Chelsea_is_so_good"
#save timeline
base_path = f"./video_data/{video_name}"
os.makedirs(base_path, exist_ok=True)
final_wav = os.path.join(base_path, "tts_timeline.wav")

timeline.export(final_wav, format="wav")

<_io.BufferedRandom name='./video_data/Pep_Guardiola_explained_why_Chelsea_is_so_good/tts_timeline.wav'>

In [3]:
from tool_language_video_translate.utils import mux_with_video

In [16]:
import tempfile
workdir = tempfile.mkdtemp(prefix="vtp_")
print(f"[workdir] {workdir}")

[workdir] /tmp/vtp_etpipam6


In [9]:
video_name = "Pep_Guardiola_explained_why_Chelsea_is_so_good"
video_path = f"./video_data/{video_name}.mp4"
audio_path = f"./video_data/origin_audio_{video_name}.wav"
output_video_path = f"./video_data/{video_name}/translated_{video_name}.mp4"
base_path = f'./video_data/{video_name}/'
#final_wav

# Save the video
mux_with_video(
    temp_path=base_path,    
    video_path=video_path,
    new_audio_path=final_wav,
    out_video_path=output_video_path,
    mode="dub" #"voiceover",
    )


[ffmpeg] ffmpeg -y -i ./video_data/Pep_Guardiola_explained_why_Chelsea_is_so_good.mp4 -i ./video_data/Pep_Guardiola_explained_why_Chelsea_is_so_good/tts_timeline.wav -map 0:v:0 -map 1:a:0 -c:v copy -c:a aac ./video_data/Pep_Guardiola_explained_why_Chelsea_is_so_good/translated_Pep_Guardiola_explained_why_Chelsea_is_so_good.mp4


ffmpeg version 7.1.1 Copyright (c) 2000-2025 the FFmpeg developers
  built with gcc 14 (GCC)
  configuration: --prefix=/usr --bindir=/usr/bin --datadir=/usr/share/ffmpeg --docdir=/usr/share/doc/ffmpeg --incdir=/usr/include/ffmpeg --libdir=/usr/lib64 --mandir=/usr/share/man --arch=x86_64 --optflags='-O2 -flto=auto -ffat-lto-objects -fexceptions -g -grecord-gcc-switches -pipe -Wall -Werror=format-security -Wno-error=incompatible-pointer-types -Wp,-U_FORTIFY_SOURCE,-D_FORTIFY_SOURCE=3 -Wp,-D_GLIBCXX_ASSERTIONS -specs=/usr/lib/rpm/redhat/redhat-hardened-cc1 -fstack-protector-strong -specs=/usr/lib/rpm/redhat/redhat-annobin-cc1 -m64 -march=x86-64 -mtune=generic -fasynchronous-unwind-tables -fstack-clash-protection -fcf-protection -mtls-dialect=gnu2 -fno-omit-frame-pointer -mno-omit-leaf-frame-pointer ' --extra-ldflags='-Wl,-z,relro -Wl,--as-needed -Wl,-z,pack-relative-relocs -Wl,-z,now -specs=/usr/lib/rpm/redhat/redhat-hardened-ld -specs=/usr/lib/rpm/redhat/redhat-annobin-cc1 -Wl,--build-id

In [15]:
from tool_language_video_translate.utils import run_ffmpeg

In [18]:
orig_wav

'./video_data/Pep_Guardiola_explained_why_Chelsea_is_so_good/orig.wav'

In [21]:
video_path

'./video_data/Pep_Guardiola_explained_why_Chelsea_is_so_good.mp4'

In [22]:
td = base_path
orig_wav = os.path.join(td, "orig.wav")
run_ffmpeg(["-i", video_path, 
            "-vn", 
            "-acodec", "pcm_s16le", 
            "-ar", "16000", 
            "-ac", "1", orig_wav])

TypeError: sequence item 2: expected str instance, list found